 **IMPORT AND SHOW DATA SAMPLE**

In [28]:
import pandas as pd
register = pd.read_csv('Customer_Registered.csv')
transaction = pd.read_csv('Customer_Transaction.csv')

<ipython-input-28-dff038ab49db>:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  register = pd.read_csv('Customer_Registered.csv')


In [29]:
print(register.head())

   ID   Contract  LocationID  BranchCode  Status created_date    stopdate
0   1  SGDN00215         8.0         1.0       0   11/25/2011    1/5/2012
1   2  SGDN00214         8.0         1.0       0    6/14/2012         NaN
2   3  SGD374348         8.0         1.0       0    11/1/2012         NaN
3   4  SGD022064         8.0         1.0       2    6/22/2011   5/29/2013
4   5  SGD041015         8.0         5.0       2   12/17/2011  11/11/2014


In [30]:
print(transaction.head())

   Transaction_ID  CustomerID Purchase_Date     GMV
0               0     1327813      6/1/2022   95000
1               1     1157830      6/1/2022   75000
2               2      873915      7/1/2022   95000
3               3     3505071      7/1/2022   90000
4               4     2930918      7/1/2022  109091


**CUSTOMER SEGMENTATION**

In [31]:
transaction['Purchase_Date'] = pd.to_datetime(transaction['Purchase_Date'])

valid_customers = register[(register['stopdate'].isna()) & (register['ID'] != 0)]

merged_df = pd.merge(valid_customers, transaction, left_on='ID', right_on='CustomerID')

reference_date = pd.to_datetime('2022-09-01')

rfm = merged_df.groupby('CustomerID').agg({
    'Purchase_Date': lambda x: (reference_date - x.max()).days,  # Recency
    'Transaction_ID': 'nunique',  # Frequency
    'GMV': 'sum'  # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'recency', 'frequency', 'monetary']

# Calculate R, F, M scores using quartiles
rfm['r_quartile'] = pd.qcut(rfm['recency'], 4, labels=False, duplicates='drop') + 1
rfm['f_quartile'] = pd.qcut(rfm['frequency'], 4, labels=False, duplicates='drop') + 1
rfm['m_quartile'] = pd.qcut(rfm['monetary'], 4, labels=False, duplicates='drop') + 1
rfm['f_quartile'] = 5 - rfm['f_quartile']

rfm['RFM_Score'] = rfm['r_quartile'].astype(str) + rfm['f_quartile'].astype(str) + rfm['m_quartile'].astype(str)

rfm['Date'] = reference_date.strftime('%Y-%m-%d')

print(rfm.head())

   CustomerID  recency  frequency  monetary  r_quartile  f_quartile  \
0       71739       62          1    105000           2           4   
1       72014       62          2    254091           2           4   
2       72052       31          1    145000           1           4   
3       72657       62          1    200000           2           4   
4       74549       62          1    125000           2           4   

   m_quartile RFM_Score        Date  
0           3       243  2022-09-01  
1           4       244  2022-09-01  
2           4       144  2022-09-01  
3           4       244  2022-09-01  
4           4       244  2022-09-01  


In [32]:
scores = rfm['RFM_Score']

In [33]:
print (scores.head())

0    243
1    244
2    144
3    244
4    244
Name: RFM_Score, dtype: object


In [34]:
def classify_customer(rfm_score):
    vip = {'444', '443', '343', '344', '333', '334', '244', '243'}
    loyal = {'221', '222', '231', '232', '241', '242', '331', '332', '341', '342', '441', '442', '421', '422'}
    potential = {'223', '224', '233', '234', '413', '414', '423', '424', '313', '314'}

    if rfm_score in vip:
        return "Nhóm khách hàng VIP"
    elif rfm_score in loyal:
        return "Nhóm khách hàng thân thiết"
    elif rfm_score in potential:
        return "Nhóm khách hàng tiềm năng"
    else:
        return "Nhóm khách hàng vãng lai"

BGC_name = rfm['RFM_Score'].apply(classify_customer)

print(BGC_name.head())

0         Nhóm khách hàng VIP
1         Nhóm khách hàng VIP
2    Nhóm khách hàng vãng lai
3         Nhóm khách hàng VIP
4         Nhóm khách hàng VIP
Name: RFM_Score, dtype: object


**RATIO & CUMULATIVE SUM OF CUSTOMER**

In [35]:
rfm_total_users = rfm.groupby('RFM_Score').size().reset_index(name='total_users')

rfm_total_users['ID'] = rfm_total_users['total_users'].rank(method='first', ascending=False)

rfm_total_users['CumulativeSum'] = rfm_total_users['total_users'].cumsum()

# Calculate ratio (%) column
total_user_sum = rfm_total_users['total_users'].sum()
rfm_total_users['ratio (%)'] = (rfm_total_users['CumulativeSum'] / total_user_sum) * 100
rfm_total_users['ratio (%)'] = rfm_total_users['ratio (%)'].round(2)

result = rfm_total_users[['RFM_Score', 'total_users', 'CumulativeSum', 'ratio (%)']]

print(result)

   RFM_Score  total_users  CumulativeSum  ratio (%)
0        141        13744          13744      12.05
1        142         4648          18392      16.12
2        143         9271          27663      24.25
3        144        14386          42049      36.86
4        241        13743          55792      48.91
5        242         4627          60419      52.96
6        243         8952          69371      60.81
7        244         3673          73044      64.03
8        341        18322          91366      80.09
9        342         6354          97720      85.66
10       343        12085         109805      96.25
11       344         4276         114081     100.00


**RATIO & CUMULATIVE SUM OF REVENUE**

In [36]:
rfm_revenue_users = rfm.groupby('RFM_Score')['monetary'].sum().reset_index(name='total_revenue')

rfm_revenue_users['ID'] = rfm_revenue_users['total_revenue'].rank(method='first', ascending=False)

rfm_revenue_users['CumulativeSum'] = rfm_revenue_users['total_revenue'].cumsum()

# Calculate ratio (%) column
total_revenue_sum = rfm_revenue_users['total_revenue'].sum()
rfm_revenue_users['ratio (%)'] = (rfm_revenue_users['CumulativeSum'] / total_revenue_sum) * 100
rfm_revenue_users['ratio (%)'] = rfm_revenue_users['ratio (%)'].round(2)

result = rfm_revenue_users[['RFM_Score', 'total_revenue', 'CumulativeSum', 'ratio (%)']]

print(result)

   RFM_Score  total_revenue  CumulativeSum  ratio (%)
0        141      973504768      973504768       8.75
1        142      392620627     1366125395      12.28
2        143      908425181     2274550576      20.45
3        144     2480266236     4754816812      42.74
4        241      980680368     5735497180      51.56
5        242      390906252     6126403432      55.07
6        243      877168721     7003572153      62.96
7        244      524794709     7528366862      67.68
8        341     1313774650     8842141512      79.49
9        342      536938514     9379080026      84.31
10       343     1184223766    10563303792      94.96
11       344      560957406    11124261198     100.00
